In [1]:
!git clone https://github.com/ruzanovad/diploma.git

Cloning into 'diploma'...
remote: Enumerating objects: 465, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 465 (delta 3), reused 7 (delta 2), pack-reused 454 (from 1)
Receiving objects: 100% (465/465), 28.96 MiB | 33.10 MiB/s, done.
Resolving deltas: 100% (256/256), done.


In [2]:
%ls

diploma/


In [3]:
%cd diploma/cnn-rnn

/kaggle/working/diploma/cnn-rnn


In [4]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

In [6]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

In [7]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/ --bind_all
# %tensorboard --logdir=your_log_dir --bind_all

In [ ]:
%run main.py --batch-size 256 --accumulate-batch 512 --train --log-step 1 \
--num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed \
--data-path /kaggle/input/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--notebook --gpu



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


==========[Train]==========


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/tensorboard.py:195: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

W0324 15:34:19.740000 64 torch/multiprocessing/spawn.py:160] Terminating process 749 via signal SIGTERM


ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/spawn.py", line 90, in _wrap
    fn(i, *args)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 173, in _wrapping_function
    results = function(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 575, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 982, in _run
    results = self._run_stage()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1024, in _run_stage
    self._run_sanity_check()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1053, in _run_sanity_check
    val_loop.run()
  File "/usr/local/lib/py

In [15]:
!nvidia-smi

Mon Mar 24 14:31:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             31W /   70W |     497MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
torch.cuda.empty_cache()

In [4]:
%run main.py --batch-size 8 --accumulate-batch 64 --train --log-step 1 \
--train-sample 10 --test-sample 2 --val-sample 2 --num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png \
--notebook

/home/kaiser/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/kaiser/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Image2Latex      | 14.5 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-----------------------------

==========[Train]==========


/home/kaiser/anaconda3/lib/python3.12/site-packages/pytorch_lightning/loggers/tensorboard.py:195: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

torch.Size([2, 3, 64, 384])


...

Validation: |          | 0/? [00:00<?, ?it/s]

torch.Size([2, 3, 64, 384])
torch.Size([8, 3, 64, 480])


Validation: |          | 0/? [00:00<?, ?it/s]

torch.Size([2, 3, 64, 384])


ZeroDivisionError: float division by zero